In [61]:
import boto3
import datetime as dt
import dateutil.relativedelta as relativedelta
import pandas as pd
import base64

In [2]:
access_creds = open('access_creds.txt', 'r')

In [3]:
creds = access_creds.readlines()

In [4]:
ec2 = boto3.client('ec2', region_name='us-east-1', aws_access_key_id=creds[0][:-1], 
             aws_secret_access_key=creds[1][:-1])

In [6]:
#ec2.describe_instances()

In [92]:
spot_launch_script = b'''
#!/bin/sh \n

cd /home/ubuntu/\n
echo 'create folder'\n

rm -r -f processing_information\n

mkdir processing_information\n
cd processing_information\n

echo 'clone code' \n
git clone https://github.com/kuhanw/seq2seq.git \n
git clone https://github.com/kuhanw/processed_data.git \n

cd processed_data \n

unrar e processed_data.rar \n
'''

In [93]:
price_history = ec2.describe_spot_price_history(InstanceTypes=['p2.xlarge'], EndTime=dt.datetime.now(), 
                                                MaxResults=100, ProductDescriptions=['Linux/UNIX'])

In [94]:
df_price_history = pd.DataFrame(price_history['SpotPriceHistory'])

In [95]:
df_price_history['SpotPrice'] = df_price_history['SpotPrice'].apply(float)

In [96]:
df_price_history['SpotPrice'].describe()

count    100.000000
mean       0.348975
std        0.162561
min        0.242700
25%        0.254700
50%        0.282500
75%        0.321050
max        0.724000
Name: SpotPrice, dtype: float64

In [ ]:
spot_launch_script = b'''
#!/bin/sh \n

cd /home/ubuntu/\n
echo 'create folder'\n

rm -r -f processing_information\n

mkdir processing_information\n
cd processing_information\n

echo 'clone code' \n
git clone https://github.com/kuhanw/seq2seq.git \n
git clone https://github.com/kuhanw/processed_data.git \n

cd processed_data \n

unrar e processed_data.rar \n

In [105]:
spot_request = ec2.request_spot_instances(
    BlockDurationMinutes=180,
    ClientToken='instance-13',
    DryRun=False,
    InstanceCount=1,
    LaunchSpecification={        
        'SecurityGroups': ['default'],
        'ImageId': 'ami-7399ec09',
        'InstanceType': 'p2.xlarge',
         'UserData': base64.b64encode(b'''#!/bin/sh\n 
         cd /home/ubuntu\n 
         echo 'create folder'\n 
         rm -r -f processing_information\n 
         mkdir processing_information\n 
         cd processing_information\n 
         mkdir chkpt\n
         echo 'clone code'\n 
         git clone https://github.com/kuhanw/seq2seq.git\n 
         git clone https://github.com/kuhanw/processed_data.git\n 
         cd processed_data\n unrar e processed_data.rar\n
         cd /home/ubuntu\n
         mount -t nfs4 -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 fs-31675578.efs.us-east-1.amazonaws.com:/ efs\n
         chmod -R 777 processing_information\n
         export PATH="/home/ubuntu/.conda/envs/tensorflow-gpu/bin:$PATH"\n
         cd processing_information/seq2seq\n
         nohup python seq2seq_training.py -s ../../efs/chkpt -r ../../efs/chkpt -cells 512 -n_layers 2 -n_embedding 2048 -dropout 0.7 -beam_length 10 -minibatch_s
ize 128 > out.log &\n''').decode("ascii"),
        'KeyName':'kuhan_aws'
    },
    SpotPrice=str(df_price_history['SpotPrice'].max()),
    Type='one-time',
    InstanceInterruptionBehavior='terminate'
)